In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

### Data Cleaning:

1.) Check for missing values and handle them appropriately.


In [ ]:
sti_df = pd.read_excel("Mini Project Shark Tank India EDA.xlsx")

In [ ]:
# Finding null values 
sti_df.isnull().sum()

In [ ]:
# Found empty string in two columns
(sti_df.applymap(lambda x: isinstance(x, str) and x.strip() == '')).sum()
# Also using conditional formating

In [ ]:
# Index positions of empty values in 'deal' column
sti_df[sti_df["deal"]== " "].index

# Replacing with 1 as they got the deal and all other data is available
sti_df.loc[[1, 16], 'deal'] = 1

In [ ]:
#  Index positions of empty values in 'equity_per_shark' column
sti_df[sti_df["equity_per_shark"]== " "].index

# Filling index-2 in "equity_per_shark" with 15 as total equity is 30 divided between 2 sharks as deal amount is same
sti_df.loc[2,"equity_per_shark"]= sti_df.loc[2,"deal_equity"] /2

# Droping the other(index-55) as the company didn't get deal and has empty value 
sti_df = sti_df.drop(index = 55)

2.)  Identify and handle any duplicate records if present.

In [ ]:
# Finding duplicates
sti_df.duplicated().sum()

# viewing those rows
sti_df[sti_df.duplicated(keep = False)]

In [ ]:
# Droping duplicates retaining 1 ocurrence
sti_df = sti_df.drop_duplicates(keep = "first")

3.)  Convert data types if necessary.

In [ ]:
# checking data types
sti_df.dtypes

# Converting the numeric values as "object" to int and float
sti_df = sti_df.astype({"deal":int ,"equity_per_shark":float})

## Analysis

#### 1) How many deals were successfully made and what percentage of total pitches did they constitute?


In [ ]:
# Number of successfull deals  (1-> success, 0-> unsuccess)
sti_df["deal"].value_counts() # total= 65 deals

# Percentage of successfull deals

round(((sti_df["deal"].sum()/sti_df.shape[0])*100),3)

#### 2) What is the average and median deal_amount for pitches that received investment?

In [ ]:
# one row showed inconsistencies between deal and deal_amount value,  deal->0 and amount-> 100, 
# means they might have got the deal,indicating input errors. These have been reviewed and corrected.

In [ ]:
sti_df["deal_amount"].sum() 
sti_df["deal_amount"].value_counts()   # Both values Is diffrent from

sti_df[(sti_df["deal"] == 1) & (sti_df["deal_amount"] != 0)]["deal_amount"].sum()
sti_df[(sti_df["deal"] == 1) & (sti_df["deal_amount"] != 0)]["deal_amount"].value_counts()

# Found this mistake from value_counts of 100
 sti_df.loc[19,'deal'] = 1 # corrected

In [ ]:
# Average
sti_df["deal_amount"].mean()

#Median
sti_df["deal_amount"].median()

#### 3) Which shark has made the highest number of investments? Provide a visualization of top 3 sharks

In [ ]:
T3Sh = sti_df[['ashneer_deal', 'anupam_deal', 'aman_deal','namita_deal', 'vineeta_deal', 'peyush_deal', 'ghazal_deal']].sum().sort_values(ascending=False).head(3)

plt.bar(T3Sh.index, T3Sh.values, color='skyblue')

plt.title('Top 3 Shark by No. of Investments', fontsize=14)
plt.xlabel('Sharks', fontsize=12)
plt.ylabel('Number of Deals', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y')

plt.show()

#### 4) What is the total amount invested by all sharks combined in entire season.


In [ ]:
sti_df['deal_amount'].sum()

#### 5) How does the deal_equity distribution look? Are there any outliers?

In [ ]:
sns.boxplot(x=sti_df['deal_equity'], color='SkyBlue')
plt.title('Boxplot of Deal Equity (%)')
plt.xlabel('Equity Percentage')

plt.show()

In [ ]:
Q1 = sti_df["deal_equity"].quantile(0.25)
Q3 = sti_df["deal_equity"].quantile(0.75)

IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = sti_df[(sti_df["deal_equity"]<lower_bound)|(sti_df["deal_equity"]>upper_bound)]

#### 6) Find the correlation between deal_valuation and ask_valuation. What insight can be drawn from this?

In [ ]:
sti_df[["ask_valuation","deal_valuation"]].corr()
#  There is no linear correlation ~0. which means sharks generally don't base their final valuation 
#  directly on the asking price

#### 7) What is the average equity percentage given to the sharks per deal?

In [ ]:
round(sti_df[sti_df["deal"] == 1]["equity_per_shark"].mean(), 3)

#### 8) Identify which episode had the highest number of deals and visualize it.

In [ ]:
deals_per_episod = sti_df[sti_df["deal"] == 1]["episode_number"].value_counts()
a = deals_per_episod.sort_values(ascending =False).head(5)
a

In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(a.index, a.values, color='skyblue')
plt.title("Number of Deals per Episode")
plt.xlabel("Episode Number")
plt.ylabel("Number of Deals")
plt.xticks(rotation=0)
plt.grid(axis='y')

plt.show()

#### 9) How many pitches asked for more than ₹1 crore and how many of those received investments?

In [ ]:
# 1) 
sti_df[sti_df["pitcher_ask_amount"] >= 100]#["pitcher_ask_amount"].count()  <-- #To get exact number

# 2) 
sti_df[(sti_df["pitcher_ask_amount"] >= 100) & (sti_df["deal"] ==1)]#["pitcher_ask_amount"].count()    <-- #To get exact number


#### 10) What percentage of pitches involved more than one shark investing together?

In [ ]:
multi_shark_deal = sti_df[(sti_df["deal"] == 1) & (sti_df["total_sharks_invested"] > 1)].shape[0]

total_shark_deal = sti_df[sti_df["total_sharks_invested"] >= 1].shape[0]

print(f"{round(((multi_shark_deal/total_shark_deal)*100),2)} %")

In [ ]:
# Found a row with input error in deal as("1") but didn't got the deal or there was no details about the deal
# So it needs to be reviewed and should be deleted or complete the data

sti_df[(sti_df["deal"] == 1) != (sti_df["total_sharks_invested"] >= 1)]

#### 11) How does the investment behavior of Ashneer Grover compare with Peyush Bansal in terms of total amount invested?

In [ ]:
ashneer_deal = sti_df[sti_df["ashneer_deal"] == 1]["amount_per_shark"].sum()
peyush_deal = sti_df[sti_df["peyush_deal"] == 1]["amount_per_shark"].sum()

amounts = [ashneer, peyush]
labels = [f'Ashneer\n₹{round(ashneer,2)}L', f'Peyush\n₹{round(peyush,2)}L']
colors = ['orange', 'skyblue']

plt.pie(amounts, labels=labels,autopct='%1.1f%%', colors=colors, startangle=140)
plt.title("Investment Amounts: Ashneer vs Peyush")

plt.show()


#### 12) Create a box plot to analyze amount_per_shark. What insights can be gathered?

In [ ]:

plt.boxplot(sti_df["amount_per_shark"], vert=False, patch_artist=True)
plt.title("Boxplot of Amount per Shark")
plt.xlabel("Amount")
plt.grid(axis='x')
plt.show()


# The median is closer to the lower quartile (Q1), and the right whisker is longer which suggests right-skewed data
# We can say most sharks invest smaller amounts, but a few invest very large amounts
# You can clearly see several outliers beyond the upper whisker
# These outliers are large investment amounts — possibly ₹60L, ₹75L, ₹100L etc.

#### 13) Are there any cases where the deal_amount exceeded the pitcher_ask_amount? If yes, list those cases.

In [ ]:
sti_df[sti_df["pitcher_ask_amount"] < sti_df["deal_amount"]][["brand_name","pitcher_ask_amount","deal_amount"]]

#### 14) Which shark has the highest return on investment (ROI) based on deal_amount vs. deal_equity?

In [ ]:
sti_df["initial_equity_value"] = (sti_df["deal_equity"] / 100) * sti_df["deal_valuation"]
sti_df["roi_initial"] = (sti_df["initial_equity_value"] / sti_df["deal_amount"]) - 1


sharks = ['ashneer_deal', 'anupam_deal', 'aman_deal', 'namita_deal',
          'vineeta_deal', 'peyush_deal', 'ghazal_deal']


roi_per_shark = {}

for shark in sharks:
    shark_df = sti_df[sti_df[shark] == 1]
    max_roi = shark_df["roi_initial"].max()
    roi_per_shark[shark.replace("_deal", "").capitalize()] = round(max_roi, 2)


sorted_roi = dict(sorted(roi_per_shark.items(), key=lambda item: item[1], reverse=True))

for shark, roi in sorted_roi.items():
    print(f"{shark}: Max ROI ={roi*100}%")


#### 15) Identify trends in equity distribution—are sharks investing in lower or higher equity stakes over time?

In [ ]:
sns.scatterplot(data=sti_df, x='episode_number', y='deal_equity', color='darkred')
plt.title('Trend of Deal Equity Over Time(Episodes)')
plt.xlabel('Episode Number')
plt.ylabel('Deal Equity (%)')
plt.grid(True)

plt.show()

# this chart shows no consistent trend of increasing or decreasing. Most deals are concentrated below 25%, 
# indicating a preference for minority stakes by shark

#### 16) What is the relationship between pitcher_ask_amount and deal_amount? Do pitchers who ask for less tend to secure more deals?

In [ ]:
sns.scatterplot(data=sti_df[sti_df["deal"]==1], x='pitcher_ask_amount', y='deal_amount', color='darkblue')
plt.title('Pitcher Ask Amount vs Deal Amount')
plt.xlabel('Pitcher Ask Amount (in Lakhs)')
plt.ylabel('Deal Amount (in Lakhs)')
plt.grid(True)

plt.show()


#### 17) Identify if there is a pattern in episode-wise deal closures—do certain episodes see more investments?

In [ ]:
episode_deal_counts = sti_df[sti_df["deal"] == 1]["episode_number"].value_counts().sort_index()

episode_deal_counts.plot(kind='bar', color='lightgreen')
plt.title('Number of Deals per Episode')
plt.xlabel('Episode Number')
plt.ylabel('Number of Deals')
plt.grid(axis='y')

plt.show()

#### 18) Analyze the impact of equity dilution on deal_valuation. Are lower equity asks leading to higher deal valuations

In [ ]:
sns.scatterplot(data=sti_df[sti_df["deal_equity"] >0], x='deal_equity', y='deal_valuation', color='darkblue')
plt.title('Equity Dilution vs Deal Valuation')
plt.xlabel('Equity %')
plt.ylabel('Deal Valuation')
plt.grid(True)

plt.show()